https://data-flair.training/blogs/python-mini-project-speech-emotion-recognition/

In [23]:
#pip install numpy scikit-learn librosa soundfile pyaudio 

In [24]:
from IPython import get_ipython;   
get_ipython().magic('reset -sf')

In [25]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import random 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

import warnings

#Ignore warnings
warnings.filterwarnings('ignore') 

In [26]:
# Set the seed for reproducibility
SEED = 80
random.seed(SEED)
np.random.seed(SEED)

In [27]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            #mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            mel=np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [28]:
#DataFlair - Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
#DataFlair - Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [29]:
#DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(".\\01-speech-emotion-data\\Actor_*\\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)  

In [30]:
%%time

#DataFlair - Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

CPU times: total: 24.7 s
Wall time: 15.1 s


In [31]:
#DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [32]:
#DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [33]:
# Gridsearch parameters
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (150,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}

In [34]:
#DataFlair - Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.001, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(500,), learning_rate='adaptive', max_iter=500)

In [35]:
# Creating a GridSearchCV object
clf = GridSearchCV(model, param_grid, n_jobs=-1, cv=5)

In [36]:
# Fitting the model with clf
clf.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=MLPClassifier(alpha=0.001, batch_size=256,
                                     hidden_layer_sizes=(500,),
                                     learning_rate='adaptive', max_iter=500),
             n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'], 'alpha': [0.001, 0.05],
                         'hidden_layer_sizes': [(50,), (100,), (150,)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']})

In [37]:
# Printing best parameter found.
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.001, 'hidden_layer_sizes': (150,), 'learning_rate': 'constant', 'solver': 'adam'}


In [38]:
#Obtaining the best parameter found. 
best_model = clf.best_estimator_

In [39]:
#Cross validation
cross_val_scores = cross_val_score(best_model, x_train, y_train, cv=5)
print(f'Cross-validation scores: {cross_val_scores}')
print(f'Mean cross-validation score: {cross_val_scores.mean()}')

Cross-validation scores: [0.75       0.73043478 0.73913043 0.77391304 0.77391304]
Mean cross-validation score: 0.7534782608695652


In [40]:
# predicting with the best model
y_pred = clf.predict(x_test)

In [41]:
# Printing accuracy 
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy with optimized hyperparameters: {accuracy}')

Accuracy with optimized hyperparameters: 0.8072916666666666


In [42]:
# DataFlair - Calculating accuracy 
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)
# DataFlair - printing accuracy 
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 80.73%


In [43]:
import joblib

# saving the model
joblib.dump(clf, ".\\01-speech-emotion-data\\Models\\model.pkl") 

['.\\01-speech-emotion-data\\Models\\model.pkl']

In [44]:
import joblib

# testing the model
clf = joblib.load(".\\01-speech-emotion-data\\Models\\model.pkl")

clf.predict(x_test)

array(['happy', 'calm', 'happy', 'happy', 'disgust', 'calm', 'calm',
       'happy', 'disgust', 'happy', 'happy', 'disgust', 'fearful',
       'happy', 'disgust', 'happy', 'calm', 'disgust', 'disgust',
       'disgust', 'disgust', 'disgust', 'disgust', 'calm', 'happy',
       'happy', 'fearful', 'happy', 'fearful', 'fearful', 'happy',
       'disgust', 'happy', 'fearful', 'happy', 'calm', 'calm', 'fearful',
       'calm', 'calm', 'happy', 'disgust', 'calm', 'calm', 'fearful',
       'disgust', 'disgust', 'calm', 'calm', 'happy', 'fearful',
       'fearful', 'fearful', 'fearful', 'happy', 'fearful', 'disgust',
       'happy', 'calm', 'calm', 'disgust', 'calm', 'happy', 'calm',
       'happy', 'calm', 'calm', 'disgust', 'disgust', 'happy', 'fearful',
       'fearful', 'fearful', 'fearful', 'fearful', 'disgust', 'happy',
       'happy', 'fearful', 'fearful', 'disgust', 'calm', 'fearful',
       'calm', 'disgust', 'happy', 'disgust', 'fearful', 'happy',
       'fearful', 'disgust', 'disgus